In [1]:
from work.processing.output import get_temporal_predictions_2
import numpy as np
from work.dataset.activitynet import ActivityNetDataset
import pprint

detection_predictions_path = '../downloads/predictions/lstm_activity_detection/v1/classes/'
classification_predictions_path = '../downloads/predictions/lstm_activity_classification/v1/classes/'

dataset = ActivityNetDataset(
    videos_path='../dataset/videos.json',
    labels_path='../dataset/labels.txt'
)

Using Theano backend.


In [2]:
video = None
for v in dataset.get_subset_videos('validation'):
    if v.video_id == 'Uw_0h2UrfyY':
        video = v
print(video.serialize())

{'subset': 'validation', 'resolution': '426x240', 'duration': 139.04, 'url': 'https://www.youtube.com/watch?v=Uw_0h2UrfyY', 'annotations': [{'segment': [19.07183775351014, 117.91560686427458], 'label': 'Ballet'}], 'num_frames': 4157}


In [3]:
video.get_video_instances(16, 0)
ground_trouth = np.array([instance.output for instance in video.instances])
print(ground_trouth)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [4]:
detection_prediction_path = detection_predictions_path + video.video_id + '.npy'
classification_prediction_path =  classification_predictions_path + video.video_id + '.npy'

class_prediction = np.load(classification_prediction_path)
detection_prediction = np.load(detection_prediction_path)

print(class_prediction)
print(detection_prediction)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 105 105 105 105 105   0   0   0   0   0   0   0   0 105 105 105 105
 105 105 105 105 105 105 105 105 105   0   0 105 105 105 105 105 105 105
 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105
 105 105 105 105   0 105 105   0 105   0   0   0   0   0   0   0   0   0
   0 177 177 177   0   0 177   0   0 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177   0   0   0   0 177   0 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177   0 177   0   0 177 177 177 177
 177 177 177 177 177   0   0 177 177 105 105 177 177 105 105 105 105 177
 177 177 105 177 177 177 177 177 177 177 177 177 177 177 177 105 105 105
 105 105 105   0   0 105   0 177 177 177 177 177   0 177 177 177 177 177
 177 177 177 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105
 105 105   0   0   0 105 105 105 105 105 105   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0 105 177 177 177 10

In [5]:
mix = np.zeros(class_prediction.shape, dtype=np.int64)
for pos in range(class_prediction.size):
    if detection_prediction[pos] == 1:
        mix[pos] = class_prediction[pos]

print(mix)

prediction = get_temporal_predictions_2(mix, fps=video.fps)
pprint.pprint(prediction)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 105 105 105 105 105   0   0   0   0   0   0   0   0 105 105 105 105
 105 105 105 105 105 105 105 105 105   0   0 105 105 105 105 105 105 105
 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105
 105 105 105 105   0 105 105   0 105   0   0   0   0   0   0   0   0   0
   0 177 177 177   0   0 177   0   0 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177   0   0   0   0 177   0 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177   0 177   0   0 177 177 177 177
 177 177 177 177 177   0   0 177 177 105 105 177 177 105 105 105 105 177
 177 177 105 177 177 177 177 177 177 177 177 177 177 177 177 105 105 105
 105 105 105   0   0 105   0 177 177 177 177 177   0 177 177 177 177 177
 177 177 177 105 105 105 105 105 105 105 105 105 105 105 105 105 105 105
 105 105   0   0   0 105 105 105 105 105 105   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0 105 177 177 177 10